## Pre-Procesing

In [11]:
from preprocessing import train_valid_test_split, combine_labels

In [12]:
SOURCE_DATA_DIR = "data/ClothingAttributeDataset/images/"
TARGET_DATA_DIR = "data/ClothingAttributeDataset/"

train_valid_test_split(SOURCE_DATA_DIR, TARGET_DATA_DIR, train_size=0.8, valid_size=0.2)

In [13]:
!ls data/ClothingAttributeDataset/train | wc -l

1484


In [14]:
!ls data/ClothingAttributeDataset/valid | wc -l

372


In [15]:
LABEL_DIR = "data/ClothingAttributeDataset/labels/"
LABELS_FILE = "data/ClothingAttributeDataset/labels.csv"

labels_df = combine_labels(LABEL_DIR, LABELS_FILE)

In [18]:
pattern_cols = [col for col in labels_df.columns if "pattern_" in col]

In [24]:
labels_df.ix["000021.jpg", pattern_cols]

pattern_spot_GT       NaN
pattern_solid_GT      NaN
pattern_graphics_GT   NaN
pattern_plaid_GT      NaN
pattern_stripe_GT     NaN
pattern_floral_GT     NaN
Name: 000021.jpg, dtype: float64

In [ ]:
# binary_columns = ['skin_exposure_GT', 'collar_GT', 'gender_GT', 'scarf_GT', 'necktie_GT',  'placket_GT']
# multi_columns = ['category_GT', 'neckline_GT', 'sleevelength_GT']
    
# color_columns = ['white_GT', 'yellow_GT', 'gray_GT', 'green_GT', 'blue_GT', 'brown_GT', 'red_GT',
#                  'cyan_GT', 'black_GT', 'purple_GT', 'many_colors_GT']
# pattern_columns = ['pattern_spot_GT', 'pattern_solid_GT', 'pattern_graphics_GT', 
#                   'pattern_plaid_GT', 'pattern_stripe_GT', 'pattern_floral_GT']

In [31]:
labels_df["pattern_GT"] = labels_df[pattern_cols].apply(lambda row: row.argmax(), axis=1)

In [39]:
import json

with open("label_values.json", 'r') as f:
    label_values = json.load(f)

In [43]:
label_values

{'category_GT': {'0': 'Shirt',
  '1': 'Sweater',
  '2': 'T-shirt',
  '3': 'Outerwear',
  '4': 'Suit',
  '5': 'Tank Top',
  '6': 'Dress'},
 'neckline_GT': {'0': 'V-shape', '1': 'Round', '2': 'Other shapes'},
 'pattern_GT': {'0': 'Floral',
  '1': 'Graphics',
  '2': 'Plaid',
  '3': 'Stripe',
  '4': 'Solid',
  '5': 'Spot'},
 'sleevelength_GT': {'0': 'No sleeves',
  '1': 'Short sleeves',
  '2': 'Long sleeves'}}

In [46]:
values_to_idx = {
    "pattern_GT": {  
        "pattern_floral_GT": 0,
        "pattern_graphics_GT": 1,
        "pattern_plaid_GT": 2,
        "pattern_stripe_GT": 3,
        "pattern_solid_GT": 4,
        "pattern_spot_GT": 5
  }
}

In [48]:
labels_df.loc[mask, "pattern_GT"].map(values_to_idx["pattern_GT"], na_action="ignore")

image
000001.jpg    1
000002.jpg    5
000003.jpg    4
000004.jpg    4
000005.jpg    4
000006.jpg    4
000007.jpg    4
000008.jpg    4
000009.jpg    5
000010.jpg    4
000011.jpg    4
000012.jpg    4
000013.jpg    4
000014.jpg    4
000015.jpg    0
000016.jpg    4
000017.jpg    4
000018.jpg    4
000019.jpg    4
000020.jpg    4
000022.jpg    5
000023.jpg    3
000024.jpg    5
000025.jpg    4
000026.jpg    4
000027.jpg    0
000028.jpg    5
000029.jpg    4
000030.jpg    4
000031.jpg    4
             ..
001827.jpg    2
001828.jpg    2
001829.jpg    2
001830.jpg    2
001831.jpg    2
001832.jpg    2
001833.jpg    2
001834.jpg    2
001835.jpg    2
001836.jpg    2
001837.jpg    2
001838.jpg    2
001839.jpg    2
001840.jpg    2
001841.jpg    2
001842.jpg    2
001843.jpg    2
001844.jpg    2
001845.jpg    2
001846.jpg    2
001847.jpg    2
001848.jpg    2
001849.jpg    2
001850.jpg    2
001851.jpg    2
001852.jpg    2
001853.jpg    2
001854.jpg    2
001855.jpg    2
001856.jpg    2
Name: pattern_GT, 

In [42]:
mask = (~pd.isnull(labels_df["pattern_GT"]))

SyntaxError: invalid syntax (<ipython-input-42-035c53a1e63b>, line 2)

## Build Models

In [1]:
from classifiers import get_pretrained_model, create_attributes_model

In [2]:
TRAIN_IMAGES_FOLDER = "data/ClothingAttributeDataset/train/"
VALID_IMAGES_FOLDER = "data/ClothingAttributeDataset/valid/"
LABELS_FILE = "data/ClothingAttributeDataset/labels.csv"

In [3]:
TARGET_COLUMNS = {
    "category_GT": 7,
    "sleevelength_GT": 3,
    "neckline_GT": 3
}

#     "pattern_GT": 

In [4]:
pretrained_features, pretrained_fc, fc_dim = get_pretrained_model("alexnet")

In [8]:
models = create_attributes_model(pretrained_features, pretrained_fc, fc_dim, TARGET_COLUMNS, "weights/",
                                LABELS_FILE, TRAIN_IMAGES_FOLDER, VALID_IMAGES_FOLDER, num_epochs=1)

Epoch 0/0
LR is set to 0.001
train Loss: 0.0283 Acc: 0.6517
valid Loss: 0.0292 Acc: 0.6805
Training completed in 0.000000m 4.595832s
Best val Acc: 0.680451
Epoch 0/0
LR is set to 0.001
train Loss: 0.0493 Acc: 0.6112
valid Loss: 0.0496 Acc: 0.6402
Training completed in 0.000000m 4.024902s
Best val Acc: 0.640187
Epoch 0/0
LR is set to 0.001
train Loss: 0.0226 Acc: 0.8382
valid Loss: 0.0230 Acc: 0.8525
Training completed in 0.000000m 6.417231s
Best val Acc: 0.852459


In [9]:
models

{'category_GT': AttributeModel (
   (model): Sequential (
     (0): Dropout (p = 0.5)
     (1): Linear (9216 -> 4096)
     (2): ReLU (inplace)
     (3): Dropout (p = 0.5)
     (4): Linear (4096 -> 4096)
     (5): ReLU (inplace)
     (6): Linear (4096 -> 7)
   )
 ), 'neckline_GT': AttributeModel (
   (model): Sequential (
     (0): Dropout (p = 0.5)
     (1): Linear (9216 -> 4096)
     (2): ReLU (inplace)
     (3): Dropout (p = 0.5)
     (4): Linear (4096 -> 4096)
     (5): ReLU (inplace)
     (6): Linear (4096 -> 3)
   )
 ), 'sleevelength_GT': AttributeModel (
   (model): Sequential (
     (0): Dropout (p = 0.5)
     (1): Linear (9216 -> 4096)
     (2): ReLU (inplace)
     (3): Dropout (p = 0.5)
     (4): Linear (4096 -> 4096)
     (5): ReLU (inplace)
     (6): Linear (4096 -> 3)
   )
 )}

In [7]:
!ls -al weights/

total 1321832
drwxrwxr-x 2 paperspace paperspace      4096 Mar 30 01:15 .
drwxrwxr-x 7 paperspace paperspace      4096 Mar 30 09:35 ..
-rw-rw-r-- 1 paperspace paperspace 218252204 Mar 30 09:35 category_GT.path
-rw-rw-r-- 1 paperspace paperspace 218252194 Mar 29 23:49 example.pth
-rw-rw-r-- 1 paperspace paperspace 218186652 Mar 30 09:35 neckline_GT.path
-rw-rw-r-- 1 paperspace paperspace 218186656 Mar 30 09:36 sleevelength_GT.path
-rw-rw-r-- 1 paperspace paperspace 218252194 Mar 30 00:27 sleevelength_GT.pth
-rw-rw-r-- 1 paperspace paperspace 262392552 Mar 23 18:10 slevelength_1.h5
